In [1]:
import os

In [2]:
%pwd

'C:\\Users\\Param\\Python-Project\\CampusX\\GitHub\\end_to_end-ML_Project_with_MLFLOW\\research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'C:\\Users\\Param\\Python-Project\\CampusX\\GitHub\\end_to_end-ML_Project_with_MLFLOW'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path  

In [6]:
from mlProject.constants import * 
from mlProject.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH, params_filepath = PARAMS_FILE_PATH, 
                 schema_filepath = SCHEMA_FILE_PATH):
        self.config_filepath = read_yaml(config_filepath)
        self.params_filepath = read_yaml(params_filepath)
        self.schema_filepath = read_yaml(schema_filepath)
        
        create_directories([self.config_filepath.artifacts_root])
        
        
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config_filepath.data_ingestion
        
        create_directories([config.root_dir])
        
        data_ingestion_config = DataIngestionConfig(
            root_dir = config.root_dir,
            source_URL= config.source_URL,
            local_data_file= config.local_data_file,
            unzip_dir= config.unzip_dir
        )
        return data_ingestion_config 

In [10]:
import os
import urllib.request as req 
import zipfile 
from mlProject import logger
from mlProject.utils.common import get_size

class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config
        
        
    def download_data(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = req.urlretrieve(url = self.config.source_URL, filename=self.config.local_data_file)
            logger.info(f"{filename} downloaded successfully! here are the details : \n {headers}")
        else:
            logger.info(f"File already exist with size of :{get_size(Path(self.config.local_data_file))}")
            
            
    def unzip_data(self):
        unzip_dir = self.config.unzip_dir
        os.makedirs(unzip_dir, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, "r") as zip_ref:
            zip_ref.extractall(unzip_dir)

## Pipeline

In [12]:
try:
    config_manager = ConfigurationManager()
    data_ingestion_config = config_manager.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_data()
    data_ingestion.unzip_data()
except Exception as e:
    raise e 


[2023-12-20 19:42:12,336: INFO: common: yaml file: config\config.yaml loaded successfully]
[2023-12-20 19:42:12,339: INFO: common: yaml file: params.yaml loaded successfully]
[2023-12-20 19:42:12,340: INFO: common: yaml file: schema.yaml loaded successfully]
[2023-12-20 19:42:12,340: INFO: common: created directory at: artifacts]
[2023-12-20 19:42:12,341: INFO: common: created directory at: artifacts/data_ingestion]
[2023-12-20 19:42:12,762: INFO: 1454582314: artifacts/data_ingestion/data.zip downloaded successfully! here are the details : 
 Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: 399A:30ABA0:D9664:169522:6582F5BB
Accept-Ranges: by